In [53]:
import pandas as pd
import numpy as np
import cx_Oracle


    
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="csebsd2db.cswg.com:1521/csebsd2_int"
)
source_cursor = source_conn.cursor()

with open('CM_full_cursor.sql', 'r') as file:
    curr_gtt_query = file.read()

cur_gttdf = pd.read_sql_query(curr_gtt_query, source_conn)


DatabaseError: ORA-28547: connection to server failed, probable Oracle Net admin error

In [44]:
# Example usage
id_generator = RequestIDGenerator()


request_id = id_generator.generate_request_id()
print("Request ID:", request_id)

Request ID: 20231113194955974001


In [20]:
cur_gttdf.replace({np.nan: None}, inplace=True)
cur_gttdf.fillna('', inplace=True)
cur_gtt = cur_gttdf.astype(str)
cur_gtt = cur_gtt.head(5)
print(cur_gtt)

  PARTY_SITE_ID STORE_NBR                 STORE_NAME STORE_STATUS  \
0         25275     20054  BLACKIE'S FARM FRESH PROD       Active   
1        376902     25317         FAIRWAY E-COMMERCE       Active   
2        376998     25401     IMPERIAL BAGS PAPER CO       Active   
3        376999     25403     N & P FOOD SERVICES IN       Active   
4        377000     25415     PLAINFIELD TOB & CANDY       Active   

  STORE_BILL_TO_FLAG STORE_SHIP_TO_FLAG ACCT_ID ACCT_NBR  \
0                  Y                  Y    6638    20054   
1                  N                  Y  269082       79   
2                  Y                  Y  248132    25401   
3                  Y                  Y  278045    25403   
4                  Y                  Y  278057    25415   

                   ACCT_NAME ACCT_STATUS  ... GS_SUPER_SERVE GS_NABISCO  \
0  BLACKIE'S FARM FRESH PROD           A  ...                             
1                    FAIRWAY           A  ...                             

In [52]:
import sys
sys.path.append(r'D:\W\EBS\Customer Denorm\Utils')
import queries as q

# Assuming your DataFrame is named 'cur_gtt'
party_site_id = cur_gtt.iloc[:, 0]  # Extract the first column

# Create a copy of the original DataFrame
cur_gtt_updated = cur_gtt.copy()

# Drop the first column from the copied DataFrame
cur_gtt_updated = cur_gtt_updated.iloc[:, 1:]

# Add the first column as the last column in the copied DataFrame
cur_gtt_updated['PARTY_SITE_ID'] = party_site_id
# Print the updated DataFrame
print(cur_gtt_updated)

# If you want to use the original column name, you can replace 'New_Last_Column_Name'
# with the actual name of your first column

# Convert DataFrame to a list of tuples
data_to_update = [tuple(row) for row in cur_gtt_updated.values]
print(data_to_update)
print(q.update_test)
# Execute the update query using the cursor
source_cursor.executemany(full_update_query, data_to_update)

# Get the number of rows updated
num_rows_updated = source_cursor.rowcount
print(f"Number of rows updated: {num_rows_updated}")

# Commit the changes
source_conn.commit()

# Close the cursor and connection
# source_cursor.close()
# source_conn.close()


  STORE_NBR                 STORE_NAME STORE_STATUS STORE_BILL_TO_FLAG  \
0     20054  BLACKIE'S FARM FRESH PROD       Active                  Y   
1     25317         FAIRWAY E-COMMERCE       Active                  N   
2     25401     IMPERIAL BAGS PAPER CO       Active                  Y   
3     25403     N & P FOOD SERVICES IN       Active                  Y   
4     25415     PLAINFIELD TOB & CANDY       Active                  Y   

  STORE_SHIP_TO_FLAG ACCT_ID ACCT_NBR                  ACCT_NAME ACCT_STATUS  \
0                  Y    6638    20054  BLACKIE'S FARM FRESH PROD           A   
1                  Y  269082       79                    FAIRWAY           A   
2                  Y  248132    25401     IMPERIAL BAGS PAPER CO           A   
3                  Y  278045    25403     N & P FOOD SERVICES IN           A   
4                  Y  278057    25415     PLAINFIELD TOB & CANDY           A   

  ACCT_CUST_CLASS_CODE  ... GS_NABISCO GS_WILL_CALL SWELL HQ_FLAG  \
0    

Number of rows updated: 5


In [51]:
import sys
import importlib
sys.path.append(r'D:\W\EBS\Customer Denorm\Utils')
import queries as q
importlib.reload(q)
user_id = -999
# Add additional columns with constant values (if needed)
set_clause = f", a.last_update_date = SYSDATE, a.last_updated_by = {user_id},last_update_login = {user_id}, a.request_id = {request_id}"
full_update_query = q.update_test.format(set_clause)
print(full_update_query)



UPDATE xxfin.xxar_customer_master a
SET a.STORE_NBR = :2, a.STORE_NAME = :3, a.STORE_STATUS = :4, a.STORE_BILL_TO_FLAG = :5, a.STORE_SHIP_TO_FLAG = :6, a.ACCT_ID = TO_NUMBER(:7), a.ACCT_NBR = :8, a.ACCT_NAME = :9, a.ACCT_STATUS = :10, a.ACCT_CUST_CLASS_CODE = :11, a.ACCT_CREATION_DATE = TO_DATE(:12, 'YYYY-MM-DD HH24:MI:SS'), a.CHAIN_NBR = TO_NUMBER(:13), a.CHAIN_NAME = :14, a.CHAIN_SHORT_NAME = :15, a.CHAIN_INACTIVE_DATE = :16, a.START_CUST_NBR = :17, a.STRT_CUST_NBR2 = :18, a.END_CUST_NBR = :19, a.END_CUST_NBR2 = :20, a.PRIVATE_LABEL_OFFSET = :21, a.SUB_AUTH_OFFST = :22, a.AD_START_DAY_OF_WEEK = :23, a.COMPANY_CODE = TO_NUMBER(:24), a.HQ_CUST_NBR = :25, a.DIVISION_NBR = TO_NUMBER(:26), a.DIVISION_NAME = :27, a.EBS_GL_CUST_GROUP_NBR = :28, a.EBS_GL_CUST_GROUP_NAME = :29, a.STORE_GROUP_NBR = :30, a.STORE_GROUP_NAME = :31, a.GROUP_BUY_NBR = TO_NUMBER(:32), a.STORE_CREATION_DATE = TO_DATE(:33, 'YYYY-MM-DD HH24:MI:SS'), a.STORE_INACTIVE_DATE = TO_DATE(:34, 'YYYY-MM-DD HH24:MI:SS'), a.STOR

In [45]:
update_query = """
UPDATE xxfin.xxar_customer_master a
SET {}
WHERE a.PARTY_SITE_ID = :1
"""

# Generate the SET clause dynamically for all columns (starting from the second column)
set_clause = ", ".join(f"a.{column} = :{i+2}" for i, column in enumerate(cur_gtt.columns[1:]))

# Add additional columns with constant values (if needed)
set_clause += f", a.last_update_date = SYSDATE, a.last_updated_by = -999, a.request_id = {request_id}"

# Format the complete update query
full_update_query = update_query.format(set_clause)
print(full_update_query)


UPDATE xxfin.xxar_customer_master a
SET a.STORE_NBR = :2, a.STORE_NAME = :3, a.STORE_STATUS = :4, a.STORE_BILL_TO_FLAG = :5, a.STORE_SHIP_TO_FLAG = :6, a.ACCT_ID = :7, a.ACCT_NBR = :8, a.ACCT_NAME = :9, a.ACCT_STATUS = :10, a.ACCT_CUST_CLASS_CODE = :11, a.ACCT_CREATION_DATE = :12, a.CHAIN_NBR = :13, a.CHAIN_NAME = :14, a.CHAIN_SHORT_NAME = :15, a.CHAIN_INACTIVE_DATE = :16, a.START_CUST_NBR = :17, a.STRT_CUST_NBR2 = :18, a.END_CUST_NBR = :19, a.END_CUST_NBR2 = :20, a.PRIVATE_LABEL_OFFSET = :21, a.SUB_AUTH_OFFST = :22, a.AD_START_DAY_OF_WEEK = :23, a.COMPANY_CODE = :24, a.HQ_CUST_NBR = :25, a.DIVISION_NBR = :26, a.DIVISION_NAME = :27, a.EBS_GL_CUST_GROUP_NBR = :28, a.EBS_GL_CUST_GROUP_NAME = :29, a.STORE_GROUP_NBR = :30, a.STORE_GROUP_NAME = :31, a.GROUP_BUY_NBR = :32, a.STORE_CREATION_DATE = :33, a.STORE_INACTIVE_DATE = :34, a.STORE_LAST_UPDATE_DATE = :35, a.LOCATION_ID = :36, a.STORE_ADDRESS1 = :37, a.STORE_ADDRESS2 = :38, a.STORE_ADDRESS3 = :39, a.STORE_ADDRESS4 = :40, a.STORE_CITY =

In [43]:
from datetime import datetime, timezone, timedelta

class RequestIDGenerator:
    def __init__(self):
        self.counter = 0

    def generate_request_id(self):
        est = timezone(timedelta(hours=-5))  # Eastern Standard Time (EST)
        timestamp = int(datetime.now(est).timestamp() * 1000)  # Convert current time in EST to milliseconds
        self.counter += 1
        human_readable_format = datetime.fromtimestamp(timestamp / 1000, est).strftime('%Y%m%d%H%M%S%f')[:-3]
        request_id = int(f"{human_readable_format}{self.counter:03d}")
        return request_id


